## Fitting models to data

In biochemical studies, we often want to infer quantiative information about reactions from experiments. This often requires fitting a quantiative model to data. In this notebook, we are going to explore how this process works, how to assess the results, and what can go wrong. 

### The reaction

Consider a binding reaction between a macromolecule $M$ and a small molecule $X$:

$$MX \rightleftarrows M + X$$

We are often interested in the dissociation constant ($K_{D}$). This is defined as:

$$K_{D} \equiv \frac{[M][X]}{[MX]}.$$

To estimate $K_{D}$, one often measures some experimental observable as a function of the concentration of $[X]$ added to solution. In a perfect experiment, one would directly measure the fraction of macromolecule bound to $X$:

$$\Theta = \frac{[MX]}{[M]+[MX]}.$$

### Developing a mathematical model

A single-site binding reaction can be described with the following simple mathematical model:

$$\Theta \sim \frac{[X]/K_{D}}{1 + [X]/K_{D}}.$$

One can rarely measure $\Theta$ directly. Instead, we often have some observable that changes when $X$ binds to $M$. Consider an extreme case where a protein is unfolded when unbound, then folds when bound. We could use circular dichroism to measure the formation of secondary structure as a function of $X$, then relate this to $\Theta$. The function would look something like the following. Our observed signal at some concentration of $X$ ($S_{obs}$) arises from a mixture of the signals from $M$ ($S_{M}$) and $MX$ ($S_{MX}$). 

$$S_{obs} \sim S_{M}(1-\Theta) + S_{MX}\Theta$$

Think about this for a moment. If the protein is all bound, $\Theta$ will be 1.0 and $S_{obs}$ will equal $S_{MX}$. If none of the protein is bound, $\Theta$ will be 0.0 and $S_{obs}$ will equal $S_{M}$. At intermediate values of $\Theta$, we will have proportional mixtures of $S_{M}$ and $S_{MX}$. 

We can rearrange this to:

$$S_{obs} \sim S_{M} + \Theta (S_{MX} - S_{M})$$

We can substitute $\Theta$ from above, giving our final model:

$$S_{obs} \sim f(K_{D},S_{M},S_{MX},[X]) = S_{M} + \frac{[X]/K_{D}}{1 + [X]/K_{D}} (S_{MX} - S_{M})$$

This function $f$ has three fit parameters ($K_{D}$, $S_{M}$ and $S_{MX}$) and one control variable ($[X]$). The parameter we care about is $K_{D}$; the other two parameters let us map binding to our signal $S_{obs}$. We can experimentally manipulate the control variable $[X]$ and measure $S_{obs}$, thus allowing us to learn about $K_{D}$. 

### Estimating the parameters of the model

We now need to find the parameters of this model that reproduce our observed data. To do so, we use [nonlinear regression](https://en.wikipedia.org/wiki/Nonlinear_regression). This is a glorified form of guess-and-check, where we try to identify values for the parameters that minimize the difference between our model and observed data. In most nonlinear regression, we minimize the *least squares difference* between our calculated and observed data. If we have measured $S_{obs}$ at $N$ different concentrations of $[X]$, we would write this as:

$$D = \sum_{i=1}^{i \le N} \Big ( f(K_{D},S_{M},S_{MX},[X]_{i}) - S_{obs,[X]_{i}}) \Big )^{2}.$$

When we minimize $D$ with respect to $K_{D}$, $S_{M}$, and $S_{MX}$, we obtain the *maximum likelihood estimate* of our parameter of interest, $K_{D}$. 




In [ ]:
#@title Initialize environment

#@markdown We recommend setting up a working directory on your google drive. This is a 
#@markdown convenient way to pass files in and out of this analysis. It will 
#@markdown also allow you to save your work. If you put `biophysics` into the form
#@markdown field below, the analyis will save all of its calculations in the 
#@markdown `biophysics` directory in MyDrive (i.e. the top directory at
#@markdown https://drive.google.com). This script will create the directory if 
#@markdown it does not already exist. If the directory already exists, any files
#@markdown that are already in that directory will be available for the analysis. 
#@markdown You could, for example, put a file called `data.csv` in `biophysics` and then
#@markdown access it as "data.csv" in all cells below.
#@markdown <br/>
#@markdown Note: Google may prompt you for permission to access the drive. 
#@markdown To work in a temporary colab environment, leave this blank. Your results
#@markdown will disappear when you close the directory. 

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

# ------------------------------------------------------------------------------
# Imports

if RUNNING_IN_COLAB:
    %pip install -q ipywidgets

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy
from scipy import optimize

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import os
import colorsys

# ------------------------------------------------------------------------------
# Environment

if RUNNING_IN_COLAB:
    
    working_dir = "/content/"

    # Select a working directory on google drive
    google_drive_directory = "" #@param {type:"string"}
    google_drive_directory = google_drive_directory.strip()

    # Set up google drive
    if google_drive_directory != "":

        from google.colab import drive
        drive.mount('/content/gdrive/')

        working_dir = f"/content/gdrive/MyDrive/{google_drive_directory}"
        os.system(f"mkdir -p {working_dir}")

    os.chdir(working_dir)
    print(f"Working directory: {os.getcwd()}/")

    print("\nCurrent directory contents:")
    print(os.getcwd())
    for f in os.listdir("."):
        print(f"    {f}")
    print()
    

# ------------------------------------------------------------------------------
# Default graph label sizing

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# ------------------------------------------------------------------------------
# Fitting functions

def binding_model_full(Kd,Sm,Smx,Xtot,Mtot):
    """
    Single site binding model (SM <--> S + M) with no assumption that Mtot is
    less than Kd. 
    
    Parameters
    ----------
    Kd : float
        dissociation constant
    Sm : float
        signal of the M form 
    Smx : float
        signal of the MX form
    Xtot : float
        total X concentration
    Mtot : float
        total M concentration

    Returns
    -------
    signal : float
        spectroscopic signal given the current [MX]/([M] + [MX])
    """
    
    a = 1
    b = -(Xtot + Mtot + Kd)
    c = Mtot*Xtot
    
    MX = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
    
    theta = MX/Mtot
    
    signal = Sm + theta*(Smx - Sm)
    
    return signal

def binding_model(Kd,Sm,Smx,Xtot):
    """
    Single site binding model (SM <--> S + M)
    
    Parameters
    ----------
    Kd : float
        dissociation constant
    Sm : float
        signal of the M form 
    Smx : float
        signal of the MX form
    X : float
        total X concentration
        
    Returns
    -------
    signal : float
        spectroscopic signal given the current [MX]/([M] + [MX])
    """
    
    alpha = Xtot/Kd
    theta = alpha/(1 + alpha)
    
    obs = Sm + theta*(Smx - Sm)
    
    return obs

def binding_model_residuals(param,x_expt,y_expt,Mtot=None):
    """
    Calculate the difference between the signal calculated from the model and
    the observed data.
    
    Parameters
    ----------
    param : list-like of floats
        model parameters (Kd, Sm, Smx)
    x_expt : list-like of floats
        Xtot values at which data were collected
    y_expt : list-like of floats
        observed signal at the Xtot values given in x_expt
    Mtot : float, optional
        total M concentration. (If specified, use full binding model; if not,
        used reduced model that assumes [M]tot << Kd
    
    Returns
    -------
    residual : np.ndarray
        difference between calculated signal and y_expt for each value of 
        x_expt
    """
    
    x_expt = np.array(x_expt)
    y_expt = np.array(y_expt)
    
    if Mtot is None:
        calc = binding_model(param[0],param[1],param[2],x_expt)
    else:
        calc = binding_model(param[0],param[1],param[2],x_expt,Mtot)
    
    residual = calc - y_expt
    
    return residual


def _get_R2(y_calc,y_expt):
    
    RSS = np.sum((y_expt - y_calc)**2)
    TSS = np.sum((y_expt - np.mean(y_expt))**2)
    
    return 1 - RSS/TSS

def _run_regresssion(fcn,
                     residual_fcn,
                     param_guesses,
                     x_expt,
                     y_expt,
                     Mtot=None,
                     ax=None,
                     residual_ax=None,
                     color="black",
                     alpha=1.0,
                     max_nfev=None,
                     label=None):
    """
    Use nonlinear regression to fit a model to data.
    
    Parameters
    ----------
    fcn : function
        function to analyze
    residual_fcn : function
        residual function
    param_guesses : list-like of floats
        model parameters
    x_expt : list-like of floats
        Xtot values at which data were collected
    y_expt : list-like of floats
        observed signal at the Xtot values given in x_expt
    Mtot : float, optional
        total M concentration. (If specified, use full binding model; if not,
        used reduced model that assumes [M]tot << Kd
    ax : matplotlib.Axis, optional
        if specified, draw the points and the fit on this ax
    residual_ax : matplotlib.Axis, optional
        if specified, draw the residual points on this axis
    color : str, default="black"
        color to use to draw series
    alpha : float, default=1.0
        opacity to give to series
    max_nfev : int, optional
        maximum number of iterations to run the regression
    label : str, optional
        label to assign to the newly drawn series
    
    Returns
    -------
    fit.x : np.ndarray
        maximum likelihood fit parameter estimates
    x_calc : np.ndarray
        values of x at which calculation is done to get smooth line
    y_calc : np.ndarray
        function values calculated at x_calc
    y_calc_expt : np.ndarray
        function values calcualated at x_expt
    """
    
    x_min = np.min(x_expt)
    x_max = np.max(x_expt)
    y_min = np.min(y_expt)
    y_max = np.max(y_expt)
    del_y = y_max - y_min
    x_calc = np.linspace(x_min*.9,x_max*1.1,100)
    
    fit = optimize.least_squares(residual_fcn,
                                 x0=param_guesses,
                                 kwargs={"x_expt":x_expt,
                                         "y_expt":y_expt,
                                         "Mtot":Mtot},
                                 max_nfev=max_nfev)
    
    y_calc = fcn(*fit.x,x_calc)
    y_calc_expt = fcn(*fit.x,x_expt)
            
    if ax is not None:
        ax.plot(x_calc,y_calc,'-',lw=2,color=color,alpha=alpha,label=label)

    if residual_ax is not None:

        residual_ax.plot(x_expt,y_calc_expt - y_expt,'o',color=color,alpha=alpha)
            
    return fit.x, x_calc, y_calc, y_calc_expt
    
def fit_data(x_expt,
             y_expt,
             y_err,
             Kd_guess,
             Sm_guess,
             Smx_guess,
             Mtot=None,
             num_bootstraps=0):
    """
    Fit a binding model to the data given in x_expt, y_expt, and y_err. Generate
    plots and potentially bootstrap to assess parameter uncertainty. Model has 
    the form: MX <--> M + X. We assume there is a signal Sm proportional to [M]
    and another signal Smx proportional to [MX]. 
    
    Parameters
    ----------
    x_expt : list-like of floats
        Xtot values at which data were collected
    y_expt : list-like of floats
        observed signal at the Xtot values given in x_expt
    y_err : list-like of floats
        standard deviation of measurements for each y_expt
    Kd_guess : float
        guess for the Kd
    Sm_guess : float
        guess for the signal of M
    Smx_guess : float
        guess for the signal of MX
    Mtot : float, optional
        total M concentration. (If specified, use full binding model; if not,
        used reduced model that assumes [M]tot << Kd 
    num_bootstraps : int, default=0
        do this many bootstrap pseudreplicate fits
    """
            
    # Get data limits
    x_min = np.min(x_expt)
    x_max = np.max(x_expt)
    y_min = np.min(y_expt)
    y_max = np.max(y_expt)
    del_y = y_max - y_min
        
    # Create plot
    fig, ax = plt.subplots(2,1,figsize=(6,12))
    
    # Create data plot
    ax[0].plot(x_expt,y_expt,'o',color="black")
    ax[0].errorbar(x_expt,y_expt,y_err,fmt='o',capsize=5,ms=0,lw=1,color="black")
    ax[0].set_xlabel("$[X]_{tot}$ ($\mu M$)")
    ax[0].set_ylabel("signal")
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    
    # Create residuals plot
    ax[1].plot((x_min,x_max),(0,0),'--',lw=2,color="gray")
    ax[1].set_xlabel("$[X]_{tot}$ ($\mu M$)")
    ax[1].set_ylabel("calc - obs")
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)

    # Figure out which model to use
    if Mtot is None:
        fcn = binding_model
    else:
        fcn = binding_model_full
    
    # Do maximum likelhood fit
    fit_param, x_calc, y_calc, y_calc_expt = _run_regresssion(fcn,
                                                              binding_model_residuals,
                                                              [Kd_guess,Sm_guess,Smx_guess],
                                                              x_expt,
                                                              y_expt,
                                                              Mtot,
                                                              ax=ax[0],
                                                              residual_ax=ax[1],
                                                              color="black")
    
    # Get fit parameters
    Kd = fit_param[0]
    Sm = fit_param[1]
    Smx = fit_param[2]
    
    R2 = _get_R2(y_calc_expt,y_expt)
    
    # Figure out y limits on residuals plot
    biggest_diff = 1.1*np.max(np.abs(y_calc_expt - y_expt))
    
    if biggest_diff < del_y/4:
        span = del_y/4
    else:
        span = biggest_diff
    
    ax[1].set_ylim(-span,span)

    
    Kd_err = None
    if num_bootstraps > 0:
    
        Kd_err = []
        for i in range(num_bootstraps):
            
            # Generate a pseudoreplicate
            this_y_expt = y_expt + np.random.normal(0,y_err,len(y_expt))

            # Generate color
            s = i/(num_bootstraps)*.9 + 0.1
            color = [s,s,1]
            
            # Plot pseudoreplicate
            ax[0].plot(x_expt,this_y_expt,'o',color=color,alpha=0.5)
            
            # Do pseudoreplicate fit
            fit_param, x_calc, y_calc, y_calc_expt = _run_regresssion(binding_model,
                                                                      binding_model_residuals,
                                                                      [Kd_guess,Sm_guess,Smx_guess],
                                                                      x_expt,
                                                                      this_y_expt,
                                                                      ax=ax[0],
                                                                      residual_ax=ax[1],
                                                                      color=color,
                                                                      alpha=0.5)
            
            # Record pseudoreplicate Kd
            Kd_err.append(fit_param[0])
            
        # Record error bit
        Kd_err = np.std(Kd_err)
        Kd_err = f" $\pm$ {Kd_err:.1f}"
        
    if Kd_err is None:
        Kd_err = ""
        
    ax[0].text(x_max*0.85,y_min + del_y*0.15,f"$K_D$: {Kd:.2f}{Kd_err} $\mu M$")
    ax[0].text(x_max*0.85,y_min + del_y*0.05,f"$R^2$: {R2:.4f}")
    
# ------------------------------------------------------------------------------
# Datasets

def read_dataset(origin):
    df = pd.read_csv(origin)
    try:
        df = df.drop("Unnamed: 0",axis=1)
    except KeyError:
        pass

    return df

df1 = read_dataset("https://raw.githubusercontent.com/harmsm/biochem-jupyter-notebooks/master/binding/raw-binding-data.csv")
df2 = read_dataset("https://raw.githubusercontent.com/harmsm/biochem-jupyter-notebooks/master/binding/raw-binding-data_1.csv")

## What is fitting?

The following cell lets you fit the model above to some toy data, with increasing numbers of steps in the regression. (Think, an increasing number of guess-and-check steps). What happens to the parameters as more steps occur? When do you think we should stop doing regression steps? 

In [ ]:
#@title Press the "Play" button on the left to run.

def plot_fitting_vs_steps(max_steps):
    
    Kd_guess = 5
    Sm_guess = 0
    Smx_guess = 2
    x_expt = np.arange(5)
    y_expt = np.array([0,1,1.5,1.6,1.7])
    y_err = 0.1*np.ones(5)

    fig, ax = plt.subplots(1,figsize=(6,6))

    # Create data plot
    ax.plot(x_expt,y_expt,'o',color="black",zorder=100)
    ax.errorbar(x_expt,y_expt,y_err,fmt='o',capsize=5,ms=0,lw=1,color="black",zorder=100)
    ax.set_xlabel("$[X]_{tot}$ ($\mu M$)")
    ax.set_ylabel("signal")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([0,2])
    
    # Do maximum likelhood fit
    for i in range(1,max_steps+1):

        color = colorsys.hsv_to_rgb(i/10,1,0.8)

        fit_param, x_calc, y_calc, y_calc_expt = _run_regresssion(binding_model,
                                                                  binding_model_residuals,
                                                                  [Kd_guess,Sm_guess,Smx_guess],
                                                                  x_expt,
                                                                  y_expt,
                                                                  ax=ax,
                                                                  color=color,
                                                                  max_nfev=i+1,
                                                                  label=f"step {i}")
        print(f"Step: {i}, Kd: {fit_param[0]:.2f}, Sm: {fit_param[1]:.2f}, Smx: {fit_param[2]:.2f}")


    ax.legend(loc="lower right")
    
#Kd_guess_slider = widgets.IntSlider(min=1,max=10,by=1,value=5,description="guess Kd (micromolar)")
max_steps_slider = widgets.IntSlider(min=1,max=10,by=1,value=1,description="max_steps")

w = widgets.interactive(plot_fitting_vs_steps,
                        #Kd_guess=Kd_guess_slider,
                        max_steps=max_steps_slider)
                        
display(w)
    

## Initial parameter guesses

The result of a fit may depend on our initial guesses for our fit parameter values. A good rule of thumb is to choose plausible first guess, as it makes it more likely you find the relevant solution. Can you make the results change by changing your parameter guesses? 

(Does this rule of thumb worry you? It should... Why?)

In [ ]:
#@title Press the "Play" button on the left to run.

Kd_guess_slider = widgets.FloatSlider(min=-100,max=100,by=5,value=10,description="guess Kd (micromolar)")
Sm_guess_slider = widgets.FloatSlider(min=-100,max=100,by=5,value=0,description="guess Sm")
Smx_guess_slider = widgets.FloatSlider(min=-100,max=100,by=5,value=2,description="guess Smx (micromolar)")

def _fit_data_wrapper(Kd_guess,Sm_guess,Smx_guess):
    
    return fit_data(x_expt=np.arange(5),
                    y_expt=np.array([0,1,1.5,1.6,1.7]),
                    y_err=0.1*np.ones(5),
                    Kd_guess=Kd_guess,
                    Sm_guess=Sm_guess,
                    Smx_guess=Smx_guess,
                    num_bootstraps=0)
    

w = widgets.interactive(_fit_data_wrapper,
                        Kd_guess=Kd_guess_slider,
                        Sm_guess=Sm_guess_slider,
                        Smx_guess=Smx_guess_slider)
                        
display(w)
    

## How do we assess fit quality?

How do we decide if our fit is good? There are two major metrics: $R^{2}$ and the *fit residuals*. 

+ $R^{2}$ measures how much scatter there is off the line. If the line drawn by our model goes through every point exactly, $R^{2}$ would be 1: the fit is perfect. If the line is far away from the points, $R^{2}$ will be close to zero. This value is shown on the top plot. 
+ The *fit residuals* determine if the model goes through the data with the same level of "goodness" as a function of our control variable ($[X]$). This is the middle plot. It is calculated as $f([X]_{i}) - S_{obs,i}$. A good fit will have *random* residuals, with a similar scatter across all values of $[X]$. If you make a histogram of these values (bottom plot), it should be normally distributed and centered at zero. 

Compare the values of $R^{2}$ and the residuals using either the "Good data" or not (click toggle). Why is the fit to the good data more valid than the fit to the not-so-good data?



In [ ]:
#@title Press the "Play" button on the left to run.


def fit_quality(good_model=True):
    
    Kd_guess = 5
    Sm_guess = 0
    Smx_guess = 2
    
    x_expt = np.linspace(0,30,61)    
    if good_model:
        y_expt = binding_model(5,0,2,x_expt) + np.random.normal(0,0.1,len(x_expt))
        y_err = 0.1*np.ones(len(x_expt))
    else:
        y_expt = x_expt**1.3 + np.random.normal(0,0.1,len(x_expt))
        y_err = 1*np.ones(len(x_expt))
    

    # Create plot
    fig, ax = plt.subplots(3,1,figsize=(6,18))
    
    # Create data plot
    ax[0].plot(x_expt,y_expt,'o',color="black")
    ax[0].errorbar(x_expt,y_expt,y_err,fmt='o',capsize=5,ms=0,lw=1,color="black")
    ax[0].set_xlabel("$[X]_{tot}$ ($\mu M$)")
    ax[0].set_ylabel("signal")
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
        
        
    # Create residuals plot
    ax[1].plot((0,30*1.1),(0,0),'--',lw=2,color="gray")
    ax[1].set_xlabel("$[X]_{tot}$ ($\mu M$)")
    ax[1].set_ylabel("calc - obs")
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    
    fit_param, x_calc, y_calc, y_calc_expt = _run_regresssion(binding_model,
                                                              binding_model_residuals,
                                                              [Kd_guess,Sm_guess,Smx_guess],
                                                              x_expt,
                                                              y_expt,
                                                              ax=ax[0],
                                                              residual_ax=ax[1])
    
    if good_model:
        ax[2].hist(y_calc_expt-y_expt,bins=np.arange(-0.3,0.38,0.08))
    else:
        ax[2].hist(y_calc_expt-y_expt)

    ax[0].text(25,1,f"R2: {_get_R2(y_calc_expt,y_expt):.4f}")


good_model_toggle = widgets.ToggleButton(
    value=True,
    description='Use good data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='If set to True, use data matching model; otherwise, use non-matching data',
    icon='check'
)
    
w = widgets.interactive(fit_quality,
                        good_model=good_model_toggle)

display(w)


## How to we assess uncertainty?

How do we get error bars on our fit results? One way to do so is using pseudoreplicate bootstrapping. Can you figure out how bootstrapping works playing with the slider below? 

In [ ]:
#@title Press the "Play" button on the left to run.      

num_bootstraps_slider = widgets.IntSlider(min=0,max=10,by=1,value=0,description="num bootstraps")

def _fit_data_wrapper(num_bootstraps):
    
    return fit_data(x_expt=np.arange(5),
                    y_expt=np.array([0,1,1.5,1.6,1.7]),
                    y_err=0.1*np.ones(5),
                    Kd_guess=5,
                    Sm_guess=0,
                    Smx_guess=2,
                    num_bootstraps=num_bootstraps)
    

w = widgets.interactive(_fit_data_wrapper,
                        num_bootstraps=num_bootstraps_slider)
                        
display(w)
    

## A real example

The following two cells show fits to experimental data collected for the binding of two different proteins to a small molecule. The researchers who published these data argue that the protein in the lower panel has a lower affinity for $X$ than the protein in the top panel. Do you believe them? Please justify your answer. What might you do next to further (or correct) their analysis? 

In [ ]:
#@title Press the "Play" button on the left to run.   
fit_data(df1.Xtot,df1.Sobs,df1.Sobs_err,100,0,1,num_bootstraps=10)

In [ ]:
#@title Press the "Play" button on the left to run.   
fit_data(df2.Xtot,df2.Sobs,df2.Sobs_err,100,0,1,num_bootstraps=10)